本项目通过抓取武汉链家二手房在售信息进行保存并进行可视化分析，利用requests和BeautifulSoup库进行抓取解析，对于得到的数据利用pandas进行整理，后期将会利用matplotlib，seaborn进行可视化分析并总结。

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
import re
import sys

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
area_list = ['jiangan','jianghan','qiaokou','dongxihu','wuchang','qingshan','hongshan','hanyang','donghugaoxin','jiangxia','caidian','huangbei','xinzhou','zhuankoukaifaqu']
area_cnlist  = ['江岸','江汉','硚口','东西湖','武昌','青山','洪山','汉阳','东湖高新','江夏','蔡甸','黄陂','新洲','沌口开发区']

In [14]:
#获取解析页面
def get_page(url,num):
    try:
        r = requests.get(url,headers=headers)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        #print('--------第%d页抓取成功--------'%num) 
        return r.text
    except:
        return None

#解析页面
def parse_page(content,area_cn):
    soup = BeautifulSoup(content,'lxml')
    
    pattern = re.compile('[0-9]{4}')  #利用正则匹配年份
    years = [i.get_text().strip().split('-')[0] for i in soup.select('div[class="positionInfo"]')]
    years_list = []
    for year in years:
        y = pattern.search(year)
        if y == None:
            years_list.append('none')
        else:
            years_list.append(y.group())
    pattern1 = re.compile('"totalPage":(.*?),".*?')
    last_page = re.findall(pattern1,str(soup))
    #查找所需二手房信息，并将查找到的数据放到字典中       
    item = {}
    item['Renovation'] = [i.get_text().strip().split('|')[4] if len(i.get_text().strip().split('|'))>4 else 'none' for i in soup.select('div[class="houseInfo"]')] #改造情况
    item['Garden'] = [i.get_text().strip().split('|')[0] for i in soup.select('div[class="houseInfo"]')]    #  房名
    item['Layout'] = [i.get_text().strip().split('|')[1] for i in soup.select('div[class="houseInfo"]')]    #户型
    item['Size'] = [i.get_text().strip().split('|')[2] for i in soup.select('div[class="houseInfo"]')]      #大小
    item['Direction'] = [i.get_text().strip().split('|')[3] for i in soup.select('div[class="houseInfo"]')] #朝向
    item['Elevator'] = [i.get_text().strip().split('|')[5] if len(i.get_text().strip().split('|'))==6 else 'none' for i in soup.select('div[class="houseInfo"]')]#电梯情况
    item['Year'] = years_list                                                                               #二手房年份
    item['Distric'] = [i.get_text().strip().split('-')[1] for i in soup.select('div[class="positionInfo"]')] # 小区
    item['Floor'] = [i.get_text().strip().split('-')[0][0:3] for i in soup.select('div[class="positionInfo"]')] #楼层
    item['Price'] = [i.get_text().strip() for i in soup.select('div[class="totalPrice"] span')]                 #价格
    item['house_unit_price'] = [i.get_text().strip() for i in soup.select('div[class="unitPrice"] span')]#均价
    item['Region'] = area_cn
    return pd.DataFrame(item)
    
                
               
    

In [17]:
def view_bar(num, total):  #显示进度条
    rate = num / total
    rate_num = int(rate * 100)
    r = '\r[%s%s]%d%%' % ("="*num, " "*(100-num), rate_num, )
    sys.stdout.write(r)
    sys.stdout.flush()

In [34]:
def main():
    filename = 'wuhanlianjia.csv'
    house_data = []
    data = pd.DataFrame()
    for area,area_cn in zip(area_list,area_cnlist):
        url = "https://wh.lianjia.com/ershoufang/%s" % area
        for  page_ing in range(1,101): 
            url_ing = url + "/pg" + str(page_ing) + '/'
            getpage = get_page(url_ing,page_ing)
            if len(getpage) > 0:
                house_info = parse_page(getpage,area_cn)
                house_data.append(house_info)   
            time.sleep(1)
            view_bar(page_ing,100)
        data = pd.concat(house_data, ignore_index = True)
        data.to_csv(filename,encoding = 'utf_8_sig', index = False)
        print('%s完毕' % area_cn)      
         

In [35]:
main()

[====================================================================================================]100%江岸完毕
[====================================================================================================]100%江汉完毕
[====================================================================================================]100%硚口完毕
[====================================================================================================]100%东西湖完毕
[====================================================================================================]100%武昌完毕
[====================================================================================================]100%青山完毕
[====================================================================================================]100%洪山完毕
[====================================================================================================]100%汉阳完毕
[====================================================================================================]100%东湖高新完

In [39]:
f = open('wuhanlianjia.csv','rb')
wuhanlianjia_df = pd.read_csv(f)
wuhanlianjia_df.head(10)

,Renovation,Garden,Layout,Size,Direction,Elevator,Year,Distric,Floor,Price,house_unit_price,Region
0,简装,阳电里,2室1厅,66.2平米,南,无电梯,2003,唐家墩,中楼层,115.0,单价17372元/平米,江岸
1,其他,育才一村,2室1厅,69.24平米,南,无电梯,1995,育才花桥,高楼层,150.0,单价21664元/平米,江岸
2,精装,吉祥谷,2室2厅,60.56平米,南,none,2005,后湖,低楼层,110.0,单价18164元/平米,江岸
3,其他,翰林紫园,2室1厅,91.49平米,西南,有电梯,2016,二七,高楼层,205.0,单价22407元/平米,江岸
4,其他,晋合金桥世家,2室2厅,95.88平米,南,有电梯,2013,后湖,高楼层,265.0,单价27639元/平米,江岸
5,其他,吉祥谷,2室1厅,44.51平米,西,none,2005,后湖,低楼层,85.0,单价19097元/平米,江岸
6,其他,东立国际,1室1厅,40.91平米,东北,有电梯,2007,二七,中楼层,101.0,单价24689元/平米,江岸
7,其他,统建同安家园,3室2厅,123.92平米,南 北,有电梯,2009,后湖,中楼层,235.0,单价18964元/平米,江岸
8,其他,新地盛世东方,3室2厅,124.01平米,南 北,有电梯,2010,后湖,低楼层,269.0,单价21692元/平米,江岸
9,其他,金岛御璟世家跃境,3室2厅,112平米,南 北,有电梯,2017,后湖,中楼层,255.0,单价22768元/平米,江岸


In [14]:
f.close()

通过浏览csv文件发现一些数据特征存在误差，后续将进一步对数据进行清洗整理，并对整理好的数据来进行可视化展示，分析二手房房价与地区，年份，楼层等特征之间关系，以及武汉各区二手房售卖数量进行总结